In [1]:
from cedarpy import is_authorized

In [20]:
from typing import Any


policies: str = """
    // can add fragment to memory when is the memory owner or has permissions
    permit(
        principal == User::"user-1",
        action == Action::"addFragment",
        resource == Memory::"memory-1"
    );
    permit(
        principal == User::"user-2",
        action == Action::"addFragment",
        resource == Memory::"memory-1"
    );
"""

entities: list[dict[str, Any]] = [
    {
        "uid": {
            "type": "Memory",
            "id": "memory-1"
        },
        "attrs": {
            "permissions": {
                "sharedWith": [{
                    "__entity": {
                        "type": "User",
                        "id": "user-2"
                    }
                }]
            },
            "created_by": {
                "__entity": {
                    "type": "User",
                    "id": "user-1"
                }
            }
        },
        "parents": [],
    }
]

request = {
    "principal": 'User::"user-2"',
    "action": 'Action::"addFragment"',
    "resource": 'Memory::"memory-1"',
    "context": {}
}

auth_result = is_authorized(request, policies, entities)
print(f"Authorization result: {auth_result.allowed}")

Authorization result: True


In [12]:
from uuid import uuid4
from src.entities.user import User
from src.entities.memory import Memory
from src.entities.account import Account
account1 = Account(id=uuid4())
account2 = Account(id=uuid4())
user1 = User(id=uuid4(), account=account1.id)
user2 = User(id=uuid4(), account=account2.id)
memory = Memory(title="Sample Memory", owner=user1.id, readers=[user2.id], created_by=user1.id)

In [2]:
memory.cedar_schema()

{'uid': {'id': 'fdfe175c-98b7-4280-ac6c-b8ae37390e88', 'type': 'Memory'},
 'attrs': {'editors': [],
  'readers': [],
  'owner': {'__entity': {'id': '93e2a393-db84-469c-8216-3ce1ed2b8f8a',
    'type': 'User'}}},
 'parents': []}

In [4]:
memory.cedar_eid()

{'id': 'fdfe175c-98b7-4280-ac6c-b8ae37390e88', 'type': 'Memory'}

In [5]:
user1.cedar_schema()

{'uid': {'id': '93e2a393-db84-469c-8216-3ce1ed2b8f8a', 'type': 'User'},
 'attrs': {'account': {'__entity': {'id': '0fddb033-469a-4451-b3ed-a9ecd9513b5c',
    'type': 'Account'}}},
 'parents': []}

In [6]:
user1.cedar_eid()

{'id': '93e2a393-db84-469c-8216-3ce1ed2b8f8a', 'type': 'User'}

In [36]:
from cedarpy import is_authorized

request = {
    "principal": user1.cedar_eid(True),
    "action": 'Action::"EditPin"',
    "resource": memory.cedar_eid(True),
    "context": {}
}
with open("src/policies.cedar", "r") as f:
    policies = f.read()
auth_result = is_authorized(request, policies, [user1.cedar_schema(), memory.cedar_schema()])
auth_result.allowed

True

In [35]:
policies

'// Policy 0: A User can create a Memory in their Account and see what Memories are in their Account\npermit (\n    principal,  // User\n    action in [Action::"CreateMemory", Action::"ListMemorySummaries"],\n    resource is Account\n) when {\n    principal has "account" && principal.account == resource\n};\n\n// Policy 1: A User can perform any action on a Memory they own\npermit (\n  principal,  // User\n  action,\n  resource is Memory\n)\nwhen { resource.owner == principal };\n\n// Policy 2: A User can see a Memory or Fragment if they are either a reader or editor\npermit (\n    principal,\n    action in [Action::"GetMemory"],\n    resource is Memory\n)\nwhen { principal in resource.readers || principal in resource.editors };\n\n// Policy 3: A User can update a Memory if they are an editor\npermit (\n    principal,\n    action in [Action::"SharedMemoryEditActions"],\n    resource is Memory\n)\nwhen { principal in resource.editors };\n'